Use this with the `cyto_39` environment (the scivision model needs a specific version of `pytorch` that isn't packaged for >3.9, i have raised a Github issue asking if they plan to update it)

`conda env create -f environment.yml`
`conda activate cyto_39`

In [34]:
import os
from scivision import load_pretrained_model, load_dataset
from dotenv import load_dotenv
import torch
import torchvision
import chromadb
import sys
sys.path.append('../')
from cyto_ml.models.scivision import prepare_image
from intake_xarray import ImageSource
load_dotenv()  # sets our object store endpoint and credentials from the .env file

True

In [26]:
dataset = load_dataset(f"{os.environ.get('ENDPOINT', '')}/metadata/intake.yml")
model = load_pretrained_model("https://github.com/alan-turing-institute/plankton-cefas-scivision")
dataset.test_image().to_dask()

/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/xarray/core/dataarray.py:1399: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  warnings.warn(
/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/intake_xarray/image.py:474: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(ds2.dims),


<xarray.DataArray (y: 24, x: 15, channel: 3)> Size: 1kB
dask.array<xarray-<this-array>, shape=(24, 15, 3), dtype=uint8, chunksize=(24, 15, 3), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) int64 192B 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * x        (x) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * channel  (channel) int64 24B 0 1 2

The scivision wrapper depends on this being an xarray Dataset with settable attributes, rather than a DataArray

Setting exif_tags: True (Dataset) or False (DataArray) is what controls this
https://docs.xarray.dev/en/stable/generated/xarray.DataArray.to_dataset.html 

https://github.com/alan-turing-institute/scivision/blob/07fb74e5231bc1d56cf39df38c19ef40e3265e4c/src/scivision/io/reader.py#L183
https://github.com/intake/intake/blob/29c8878aa7bf6e93185e2c9639f8739445dff22b/intake/__init__.py#L101

But now we're dependent on image height and width metadata being set in the EXIF tags to use the `predict` interface, this is set in the model description through `scivision`, this is brittle

https://github.com/alan-turing-institute/plankton-cefas-scivision/blob/main/resnet50_cefas/model.py#L71


A quick look at the example dataset that comes with the model, for reference

In this case we don't want to use the `predict` interface anyway (one of N class labels) - we want the features that go into the last fully-connected layer (as described here https://stackoverflow.com/a/52548419)

In [5]:
network = torch.nn.Sequential(*(list(model._plumbing.model.pretrained_model.children())[:-1]))

In [9]:
imgs = dataset.test_image().to_dask()
i= imgs.to_numpy()
i.shape


(24, 15, 3)

https://github.com/alan-turing-institute/plankton-cefas-scivision/blob/main/resnet50_cefas/data.py 


Pass the image through our truncated network and get some embeddings out

In [11]:
o = torch.stack([torchvision.transforms.ToTensor()(i)])
feats = network(o)
feats.shape

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 2048, 1, 1])

In [12]:
embeddings = list(feats[0].squeeze(1).squeeze(1).detach().numpy().astype(float))

In [13]:
embeddings

[0.18681475520133972,
 0.0,
 0.39956235885620117,
 0.0,
 0.09887093305587769,
 0.0,
 0.0,
 0.04044981300830841,
 0.41290155053138733,
 0.5317279100418091,
 1.0021417140960693,
 0.31051135063171387,
 0.0022781193256378174,
 0.0,
 0.0,
 0.07445969432592392,
 0.0,
 0.0,
 0.865257203578949,
 0.09340490400791168,
 0.0,
 2.9403414726257324,
 0.0,
 0.0,
 0.279595285654068,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.92582368850708,
 0.0,
 1.144113302230835,
 1.0001882314682007,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0007738349377177656,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.879579782485962,
 0.0,
 0.0,
 0.29063326120376587,
 0.0,
 0.0,
 0.5436230897903442,
 0.6394219994544983,
 1.206394910812378,
 0.0,
 0.0861923098564148,
 0.04212423786520958,
 0.0,
 1.79121732711792,
 0.0,
 0.0,
 0.0,
 1.24888277053833,
 0.0,
 0.0,
 0.0,
 0.42207008600234985,
 0.04022020101547241,
 0.0,
 0.0,
 0.2464810013771057,
 1.0166819095611572,
 1.4085975885391235,
 0.0,
 0.7129244208335876,
 0.5857464671134949,
 0.0,
 0.0,
 0.

In [14]:
print(set([type(x) for x in embeddings]))

{<class 'numpy.float64'>}


There's been some guesswork up to this point (honestly expected the input would be resized, and concerned it's not normalised) but we have a set of embeddings, at which point pop them in a vector database and see if similarity queries yield any kind of meaningful results, iterate from there

https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/ - langchain has a few, Milvus would be another option
https://python.langchain.com/v0.1/docs/integrations/vectorstores/ - there are so many integrations!
https://www.trychroma.com/ - this looks straightforward to approach

In [15]:
client = chromadb.PersistentClient(path="./vectors")
collection = client.create_collection(
        name="test_collection",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [18]:
collection.add(
    documents=["test_image"],
    embeddings=[embeddings],
    metadatas=[{"useful": "maybe"}],
    ids=["id2"]  # must be unique, are they required?
)

In [19]:
collection.get('id2',include=["embeddings"])

{'ids': ['id2'],
 'embeddings': [[0.18681475520133972,
   0.0,
   0.39956235885620117,
   0.0,
   0.09887093305587769,
   0.0,
   0.0,
   0.04044981300830841,
   0.41290155053138733,
   0.5317279100418091,
   1.0021417140960693,
   0.31051135063171387,
   0.0022781193256378174,
   0.0,
   0.0,
   0.07445969432592392,
   0.0,
   0.0,
   0.865257203578949,
   0.09340490400791168,
   0.0,
   2.9403414726257324,
   0.0,
   0.0,
   0.279595285654068,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.92582368850708,
   0.0,
   1.144113302230835,
   1.0001882314682007,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0007738349377177656,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.879579782485962,
   0.0,
   0.0,
   0.29063326120376587,
   0.0,
   0.0,
   0.5436230897903442,
   0.6394219994544983,
   1.206394910812378,
   0.0,
   0.0861923098564148,
   0.04212423786520958,
   0.0,
   1.79121732711792,
   0.0,
   0.0,
   0.0,
   1.24888277053833,
   0.0,
   0.0,
   0.0,
   0.4220700860

In [74]:
index = dataset.plankton().to_dask().compute()

In [47]:
index


,Filename
npartitions=1,
,string
,...


In [37]:
def flat_embeddings(features: torch.Tensor):
    return list(features[0].squeeze(1).squeeze(1).detach().numpy().astype(float))

In [84]:
def file_embeddings(row):
    image_data = ImageSource(row.Filename).to_dask()
    embeddings = flat_embeddings(network(prepare_image(image_data)))
    collection.add(
        documents=[row.Filename],
        embeddings=[embeddings],
        ids=[row.Filename]  # must be unique, are they required?
    )



In [30]:
from intake_xarray import ImageSource

Because all the images have slightly different dimensions as they come out of the FlowCam, we can't batch them
Push them through the model one by one and either build a list of `(id, [embeddings])` pairs, or potentially pop them straight into chromadb as we apply the function, which would keep it more dasklike?

This scales ok at 8000 or so images

In [43]:
collection.count()

74

In [85]:
res = index.apply(file_embeddings, axis=1)

Filename    https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untag...
Name: 0, dtype: object


/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/xarray/core/dataarray.py:1399: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  warnings.warn(
/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/intake_xarray/image.py:474: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(ds2.dims),
Add of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_1.tif
Insert of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_1.tif


Filename    https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untag...
Name: 1, dtype: object


Add of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_10.tif
Insert of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_10.tif


Filename    https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untag...
Name: 2, dtype: object


...

Add of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_10029.tif
Insert of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_10029.tif
Add of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_1003.tif
Insert of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_1003.tif


Filename    https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untag...
Name: 36, dtype: object
Filename    https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untag...
Name: 37, dtype: object


Add of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_10030.tif
Insert of existing embedding ID: https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untagged-images/19_10_Tank22_10030.tif


Filename    https://fw-plankton-o.s3-ext.jc.rl.ac.uk/untag...
Name: 38, dtype: object


KeyboardInterrupt: 

In [87]:
collection.count()

74

In [32]:
i = ImageSource(index.loc[0].Filename).to_dask()
i

/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/xarray/core/dataarray.py:1399: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  warnings.warn(
/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/intake_xarray/image.py:474: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(ds2.dims),


<xarray.DataArray (y: 24, x: 15, channel: 3)> Size: 1kB
dask.array<xarray-<this-array>, shape=(24, 15, 3), dtype=uint8, chunksize=(24, 15, 3), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) int64 192B 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * x        (x) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * channel  (channel) int64 24B 0 1 2

In [36]:
network(prepare_image(i))

/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/xarray/core/dataarray.py:1399: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  warnings.warn(
/home/jowals/miniconda3/envs/cyto_39/lib/python3.9/site-packages/intake_xarray/image.py:474: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(ds2.dims),


tensor([[[[0.1368]],

         [[0.1237]],

         [[0.0324]],

         ...,

         [[0.0000]],

         [[0.3849]],

         [[0.0000]]]], grad_fn=<MeanBackward1>)